<a href="https://colab.research.google.com/github/ramalamadingdong/yolo-rb3gen2-trainer/blob/main/Demo_YOLO_to_RB3Gen2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Retrain YOLOv8 on Synthetic Data

In [ ]:
!git clone https://github.com/ramalamadingdong/yolo-rb3gen2-trainer
%cd yolo-rb3gen2-trainer
!pip install -r requirements.txt

Cloning into 'yolo-rb3gen2-trainer'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 0), reused 14 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 125.14 KiB | 10.43 MiB/s, done.
/content/yolo-rb3gen2-trainer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [5]:
from ultralytics import YOLO
import os
import torch
import yaml
import gc
import random
import numpy as np
from PIL import Image, ImageDraw
from pathlib import Path

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
#The following functions are used to prepare data.

def create_directories(output_dir):
    """Create necessary output directories."""
    output_dir = Path(output_dir)
    train_dir = output_dir / 'train'
    val_dir = output_dir / 'val'
    for split in [train_dir, val_dir]:
        (split / 'images').mkdir(parents=True, exist_ok=True)
        (split / 'labels').mkdir(parents=True, exist_ok=True)
    return output_dir, train_dir, val_dir

def add_training_picture(draw, image_size):
    """Place an training image with random position and rotation."""

    train_img_pth = random.choice(list(Path('/content/yolo-rb3gen2-trainer/training_images').glob('*.png')))
    tag = Image.open(train_img_pth).convert('L')  # Convert to grayscale for better quality

    # YOU MAY WANT TO CHANGE THIS FOR BETTER RESULTS
    # Randomly resize tag between 40-120 pixels while maintaining aspect ratio
    tag_w, tag_h = tag.size
    target_size = random.randint(40, 120)
    scale = min(target_size/tag_w, target_size/tag_h)
    new_size = (int(tag_w * scale), int(tag_h * scale))

    # Use nearest neighbor resampling to preserve sharp edges and binary nature of Training picture
    tag = tag.resize(new_size, Image.NEAREST)

    # Threshold to ensure pure black and white
    tag = tag.point(lambda x: 0 if x < 128 else 255, '1')

    # Get new dimensions after resize
    width, height = tag.size

    # Random position
    x = random.randint(0, image_size[0] - width)
    y = random.randint(0, image_size[1] - height)

    # Random rotation angle
    angle = random.uniform(0, 360)

    # Apply rotation
    tag = tag.rotate(angle, expand=False)

    # Get rotated dimensions
    rot_width, rot_height = tag.size

    # Paste rotated tag onto image
    draw._image.paste(tag, (x - (rot_width - width)//2, y - (rot_height - height)//2))

    # Calculate corners of rotated rectangle
    center_x = x + width/2
    center_y = y + height/2

    # Return corners for bounding box calculation
    corners = [
        (x, y),
        (x + rot_width, y),
        (x + rot_width, y + rot_height),
        (x, y + rot_height)
    ]
    return corners

def create_yolo_label(shape, class_id, image_size):
    """Convert shape coordinates to YOLO format."""
    x_coords = [p[0] for p in shape]
    y_coords = [p[1] for p in shape]
    x_center = (min(x_coords) + max(x_coords)) / (2 * image_size[0])
    y_center = (min(y_coords) + max(y_coords)) / (2 * image_size[1])
    width = (max(x_coords) - min(x_coords)) / image_size[0]
    height = (max(y_coords) - min(y_coords)) / image_size[1]

    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

def generate_dataset(output_dir, num_images, image_size):
    """Generate synthetic dataset with images and labels."""
    output_dir, train_dir, val_dir = create_directories(output_dir)
    classes = ['AprilTag']

    # Split into train and validation
    train_size = int(0.8 * num_images)
    val_size = num_images - train_size

    for split, size in [('train', train_size), ('val', val_size)]:
        print(f"Generating {split} dataset...")
        for i in range(size):
            # Load base image
            image = Image.open('/content/yolo-rb3gen2-trainer/base_image.jpg').resize(image_size)
            draw = ImageDraw.Draw(image)

            # Generate random number of shapes
            num_shapes = random.randint(1, 3)
            labels = []

            for _ in range(num_shapes):
                skewed_corners = add_training_picture(draw, image_size)

                # Calculate bounding box from skewed corners with padding
                x_coords = [corner[0] for corner in skewed_corners]
                y_coords = [corner[1] for corner in skewed_corners]

                # Add 20% padding to bounding box
                padding_x = (max(x_coords) - min(x_coords)) * 0.1
                padding_y = (max(y_coords) - min(y_coords)) * 0.1

                bbox = [
                    (max(0, min(x_coords) - padding_x), max(0, min(y_coords) - padding_y)),
                    (min(image_size[0], max(x_coords) + padding_x),
                     min(image_size[1], max(y_coords) + padding_y))
                ]

                label = create_yolo_label(bbox, classes.index("AprilTag"), image_size)
                labels.append(label)

            # Save image
            image_path = output_dir / split / 'images' / f'image_{i:04d}.jpg'
            image.save(image_path)

            # Save label
            label_path = output_dir / split / 'labels' / f'image_{i:04d}.txt'
            with open(label_path, 'w') as f:
                f.write('\n'.join(labels))

def create_dataset_yaml(output_dir):
    """Create YAML file for dataset configuration."""
    classes = ['AprilTag']
    yaml_content = {
        'path': str(output_dir),
        'train': '/content/data/train/images',
        'val': '/content/data/val/images',
        'nc': len(classes),
        'names': classes
    }

    with open(output_dir / 'dataset.yaml', 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)

In [ ]:
#The following functions are used to Train Yolo.

def load_config(config_path: str = "/content/yolo-rb3gen2-trainer/config.yaml"):
    """
    Load configuration from YAML file.

    Args:
        config_path (str): Path to the config.yaml file

    Returns:
        dict: Configuration dictionary
    """
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

def clear_gpu_memory():
    """Clear GPU memory and garbage collection"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def train_yolo(
    data_yaml_path: str,
    config_path: str = "/content/yolo-rb3gen2-trainer/config.yaml",
    **kwargs
):
    """
    Train a YOLOv8 model on a custom dataset using configurations from config.yaml.

    Args:
        data_yaml_path (str): Path to the data.yaml file containing dataset configuration
        config_path (str): Path to the config.yaml file containing training configuration
        **kwargs: Additional arguments to override config settings
    """
    # Clear GPU memory before starting
    clear_gpu_memory()

    # Load configuration
    config = load_config(config_path)

    # Get model configuration
    model_config = config['model']
    model_type = f"{model_config['type']}.pt"

    # Get training configuration
    train_config = config['train']

    # Update training config with any provided kwargs
    train_config.update(kwargs)

    # Force CPU device if CUDA is causing issues
    if torch.cuda.is_available():
        try:
            # Print CUDA device information
            print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
            print(f"CUDA version: {torch.version.cuda}")
            print(f"PyTorch version: {torch.__version__}")
            print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

            # Test CUDA compatibility with smaller batch size
            test_model = YOLO(model_type)
            test_model.to('cuda')
            test_model.predict('https://ultralytics.com/images/bus.jpg', device='cuda', batch=1)

            # Clear memory after test
            clear_gpu_memory()

        except Exception as e:
            print(f"CUDA compatibility issue detected: {str(e)}")
            print("Falling back to CPU.")
            train_config['device'] = 'cpu'
    else:
        print("CUDA is not available. Using CPU.")
        train_config['device'] = 'cpu'

    # Initialize model
    model = YOLO(model_type)

    # Train the model with error handling
    try:
        results = model.train(
            data=data_yaml_path,
            epochs=train_config['epochs'],
            batch=train_config['batch_size'],
            imgsz=train_config['imgsz'],
            device=train_config['device'],
            project=train_config['project'],
            name=train_config['name'],
            exist_ok=train_config['exist_ok'],
            pretrained=train_config['pretrained'],
            optimizer=train_config['optimizer'],
            verbose=train_config['verbose'],
            seed=train_config['seed'],
            deterministic=train_config['deterministic'],
            single_cls=train_config['single_cls'],
            rect=train_config['rect'],
            cos_lr=train_config['cos_lr'],
            close_mosaic=train_config['close_mosaic'],
            resume=train_config['resume'],
            amp=train_config['amp'],
            fraction=train_config['fraction'],
            nbs=train_config['nbs'],
            overlap_mask=train_config['overlap_mask'],
            mask_ratio=train_config['mask_ratio'],
            dropout=train_config['dropout'],
            val=train_config['val']
        )

        return results

    except Exception as e:
        print(f"Training error occurred: {str(e)}")
        # Clear GPU memory after error
        clear_gpu_memory()
        raise e

In [ ]:
# Configuration
output_dir = 'data'
num_images = 100
image_size = (1280, 720)

# Generate dataset
generate_dataset(output_dir, num_images, image_size)

# Create dataset YAML
create_dataset_yaml(Path(output_dir))

print("Dataset generation completed!")

Generating train dataset...
Generating val dataset...
Dataset generation completed!


In [ ]:
# Path to your data.yaml file
data_yaml = "/content/data/dataset.yaml"
# Train the model using configurations from config.yaml
results = train_yolo(
    data_yaml_path=str(data_yaml)
)

In [ ]:
# Display training results
print("Training completed!")
print(f"Results saved to: {results.save_dir}")

Training completed!
Results saved to: runs/train/exp


# Next Steps: Preperation / Quantization of the model for RB3Gen2
I recommend the following be run on your personal machine but you can do this on the Colab too however the last step takes forever to complete.

In [ ]:
!pip install qai-hub-models[yolov8-det-quantized]

In [ ]:
# Create an Account on AI Hub and get API Token.
!qai-hub configure --api_token <YOUR_API_TOKEN>

In [ ]:
!python -m qai_hub_models.models.yolov8_det_quantized.export --device="RB3 Gen 2 (Proxy)" --ckpt-name /content/runs/train/exp/weights/best.pt

/usr/local/lib/python3.11/dist-packages/qai_hub_models/utils/quantization.py:132: UserWarning: 

!!! WARNING !!!
Quantized model package yolov8_det_quantized is deprecated. Use the equivalent unquantized model package (yolov8_det) instead.
You can use qai_hub_models.models.yolov8_det.export and qai_hub_models.models.yolov8_det.evaluate with the `--precision w8a8` flag to replicate previous behavior of those scripts.


  warnings.warn(
Quantizing model yolov8_det.
Uploading tmpc20gd6g1.pt
100% 12.1M/12.1M [00:02<00:00, 5.89MB/s]
Scheduled compile job (jp1y2x22p) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp1y2x22p/

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/.qaihm/fiftyone/coco-2017/train' if necessary
Found annotations at '/root/.qaihm/fiftyone/coco-2017/raw/instances_train2017.json'
INFO:fiftyone.utils.coco:Found annotations at '/root/.qaihm/fiftyone/coco-2017/raw/instances_train2017.json'
642 images found; downloading the

# RB3Gen2 Preperation / RB3Gen2 Commands


In [4]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="PATH_TO_FILE_FROM_AIHUB")
interpreter.allocate_tensors()

# Get output tensor details
output_details = interpreter.get_output_details()

q_scales = []
q_zero_points = []

for output in output_details:
    params = output['quantization_parameters']

    # Check if the tensor is quantized
    if output['quantization_parameters']['quantized_dimension'] == 0:  # Per-tensor quantization
        scale = params['scales'][0] if params['scales'].size > 0 else 1.0
        zero_point = params['zero_points'][0] if params['zero_points'].size > 0 else 0.0
    else:
        # Handle per-channel quantization (unlikely for YOLOv8)
        scale = 1.0
        zero_point = 0.0

    q_scales.append(scale)
    q_zero_points.append(float(zero_point))

# Format the constants
print(f'YOLOv8,q-offsets=<{", ".join(map(str, q_zero_points))}>, q-scales=<{", ".join(map(str, q_scales))}>')

YOLOv8,q-offsets=<36.0, 0.0, 0.0>, q-scales=<3.9993956, 0.003825209, 1.0>


In [2]:
!gst-launch-1.0 -e --gst-debug=2 \
qtiqmmfsrc name=camsrc ! video/x-raw\(memory:GBM\),format=NV12,width=1280,height=720,framerate=30/1,compression=ubwc ! queue ! tee name=split \
split. ! queue ! qtivcomposer name=mixer ! queue ! waylandsink fullscreen=true \
split. ! queue ! qtimlvconverter ! queue ! qtimltflite delegate=external external-delegate-path=libQnnTFLiteDelegate.so \
external-delegate-options="QNNExternalDelegate,backend_type=htp;" model=/opt/yolov8_det_quantized.tflite ! queue ! \
qtimlvdetection threshold=50.0 results=10 module=yolov8 labels=/opt/coco_labels.txt constants="YOLOv8,q-offsets=<21.0, 0.0, 0.0>,q-scales=<3.093529462814331, 0.00390625, 1.0>;" ! \
video/x-raw,format=BGRA,width=640,height=360 ! queue ! mixer.


/bin/bash: line 1: gst-launch-1.0: command not found
